Importing Libraries

In [1]:
import re
import nltk
import warnings
import unicodedata
import numpy as np
import pandas as pd
nltk.download('wordnet')
from fuzzywuzzy import fuzz
warnings.filterwarnings('ignore')
from nltk.stem import WordNetLemmatizer
from sklearn.cluster import AffinityPropagation
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zafreen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\zafreen\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df1=pd.read_csv('C:/Users/zafreen/Downloads/ENS/Prev_Emp_Starts.csv')
list_uni=df1['Previous Employer'].unique().tolist()
df1=pd.DataFrame(list_uni,columns=['Company'])         
df1=df1.reset_index(drop=True)
df1['Company']=df1['Company'].astype(str)
df1.head(93)
df1.shape

(9280, 1)

In [3]:
import string
vendor_stopwords=['biz', 'bv', 'co', 'comp', 'company','com', 
                'corp','corporation', 'dba', 
                'inc', 'incorp', 'incorporat', 
                'incorporate', 'incorporated', 'incorporation', 
                'international', 'intl', 'intnl', 
                'limited' ,'llc', 'ltd', 'llp', 
                'machines', 'pvt', 'pte', 'private', 'unknown','Agency','and','Assn','Assoc','Associates','Association','Bank','BV','Co','Comp','Company','Corp',
             'Corporation', 'Corpn','Enterprises','Gmbh','Group','Hotel','Hotels','Infotech', 'Info', 'infotech',
             'info','info tech', 'Info tech', 'Info Tech', 'Inc','Incorporated','International','Intl','Limited',
             'LLC','LLP','LP','Ltd','Manufacturing','Mfg','PA','PC','Pharmacy','PLC','PLLC','Restaurant','SA',
             'Sales','Service','Services','Store','Svcs','Travel','Unlimited', 'agency','and','assn','assoc',
             'associates','association','bank','bv','co','comp','company','corp','corporation', 'corpn',
             'enterprises','gmbh','group','hotel','hotels','inc','incorporated','international','intl','limited',
             'llc','llp','lp','ltd','manufacturing','mfg','pa','pc','pharmacy','plc','pllc','restaurant','sa',
             'sales','service','services','store','svcs','travel','unlimited']
#vinod
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z\s]+' if not remove_digits else r'[^a-zA-z0\s]+'
    #use reqex
    #https://dataschool.com/how-to-teach-people-sql/how-regex-works-in-sql/
    text = re.sub(pattern, '', text)
    return text
def remove_stopwords(text):
    text=" ".join([char for char in text if char not in vendor_stopwords])
    #concat
    return text
def tokenise(text):
    text=re.split("\W+",text)
    #string split
    return text

df1['Company']=df1['Company'].apply(lambda x :remove_special_characters(x) )
df1['Company']=df1['Company'].apply(lambda x: x.lower())
df1['Company']=df1['Company'].apply(lambda x :x.strip() )
df1['Company']=df1['Company'].apply(lambda x :x.strip("[]") )
df1['Company']=df1['Company'].apply(lambda x:tokenise(x))
df1['Company']=df1['Company'].apply(lambda x:remove_stopwords(x))
df1['Company']=df1['Company'].apply(lambda x:tokenise(x))
#run all functions
df1=df1.dropna()
df1.head(95)

,Company
0,"[iqvia, biotech]"
1,"[infor, geac, computer]"
2,"[livestyle, sfx, entertainment]"
3,"[k, line, america]"
4,[gotjunk]
...,...
90,"[adc, telecommunication, adc]"
91,"[adesa, kar, auction]"
92,"[adp, automatic, data, processing]"
93,"[adp, automatic, data, processing, adp]"


In [4]:
df1=df1.dropna()
#select with where
df1=df1[df1['Company'] !='[]']
#select with where
df1=df1.reset_index(drop=True)
df1['Company']=df1.sort_values(by="Company")[4:]
#sort
df1=df1.dropna()
#select with where
df1=df1.reset_index(drop=True)

In [5]:
for i in range(len(df1)):
    #vinod
    if len(df1['Company'][i])>1:
        df1.loc[i,'Company']=df1['Company'][i][0]+" "+df1['Company'][i][1]
    else:
        df1.loc[i,'Company']=df1['Company'][i][0]

# Steps in details

In [ ]:
# Given : letters,first_letter,to_remove are lists
#         to_replace is a dictionary with key value pair.
        
# Step 1: for every character in query it checks if it is alphabet. if yes it will store in a list called letters
#         Eg: query="Oracle 3Corp" letters="Oracle Corp".
# Step 2: it checks the length of query. if it is  equal to 1 then append 3 zeros to query
#         Eg: query="a" then query should be changed to "a000".
# Step 3: create a list called to_remove=('a', 'e', 'i', 'o', 'u', 'y', 'h', 'w')
# Step 4: assign first letter of letters list in a variable called first_letter
# Step 5: delete first letter from letters list
# Step 6: remove the characters from letters if they are among the characters in to_remove
# Step 7: Now check the length of letters list if it is qqual to zero them return 3 zerots as output of soundex
# Step 8: else continue
# Step 9: create a list called to_replace ={('b'): 1,('p'):2, ('c'):3, ('g'):4, ('j'):5, ('k'):6, ('q'):7, ('s'):8, ('z'): 9,
#                    ('d'):11, ('t'):12, ('l'): 13, ('m'):14, ('r',): 15,('f'):16,('v'):17,('x'):18, ('n'): 19}
# Step 10: attach first_letter to corresponding code created from remaining alphabet after removing vowels from to _remove
#          Eg: "Amazon" step1: choose "A",step2: remove a,a,o to get "mzn", step3 code m,z,n,step4 attach code to A="A14919"
# Step 13:for index,char in letters
#         if index is equal to
#         (length of letters)-1 or (index+1is less than len(letters) and char is not equal to letters of index+1)
#         then save such characters in letters
# Step 14:if first_letter is equal to first letter of list letters then assign first_letter of query as first letter of letters 
# Step 15:else enter the first value o query as first value of letters.
# Step 16: repeat Step 4 and Step 5
# Step 17: for every character in letters check if it is an integer and store it back in letters 
# Step 18: take only 3 characters of letters.
# Step 19: while length of letters is less than 3 ,Append 3 zeros if result contains less than 3 digits.
# Step 20: insert first_letter as first value of letters 
# Step 21: join each value in letters with no space in between and return the value as output of the function.

In [8]:
#define soundex
def soundex(query: str):
    #Clean up the query string
    #query = query
    letters = [char for char in query if char.isalpha()]
    # If query contains only 1 letter, return query+"000" (Refer step 5)
    if len(query) == 1:
        #if else
        return query + "000"
    to_remove = ('a', 'e', 'i', 'o', 'u', 'y', 'h', 'w')    
    # Save the first letter. Remove all occurrences of a, e, i, o, u, y, h, w.
    first_letter = letters[0]
    #assign 1st char 
    letters = letters[1:]
    # assign rest char
    letters = [char for char in letters if char not in to_remove]
              # (select char from letters where letters not in(select * from to_remove)
    if len(letters) == 0:
        #if else
        return first_letter + "000"
    # Step 2: Replace all consonants (include the first letter) with digits according to rules
    to_replace ={('b'): 1,('p'):2, ('c'):3, ('g'):4, ('j'):5, ('k'):6, ('q'):7, ('s'):8, ('z'): 9,
                   ('d'):11, ('t'):12, ('l'): 13, ('m'):14, ('r',): 15,('f'):16,('v'):17,('x'):18, ('n'): 19}
    first_letter = [value if first_letter else first_letter for group, value in to_replace.items() if first_letter in group]
    letters = [value if char else char
               for char in letters
               for group, value in to_replace.items()
               if char in group]
    #Replace all adjacent same digits with one digit.
    letters = [char for ind, char in enumerate(letters)
               if (ind == len(letters) - 1 or (ind+1 < len(letters) and char != letters[ind+1]))]
    # If the saved letter’s digit is the same the resulting first digit, remove the digit (keep the letter)
    if first_letter == letters[0]:
        letters[0] = query[0]
    else:
        letters.insert(0, query[0])
    # Remove all except first letter and 3 digits after it.
    first_letter = letters[0]
    letters = letters[1:]
    letters = [char for char in letters if isinstance(char, int)][0:3]
    # Append 3 zeros if result contains less than 3 digits.
    while len(letters) < 3:
        letters.append(0)
    #append in sql
    letters.insert(0, first_letter)
    string = "".join([str(l) for l in letters])
    #concate
    return string

In [1]:
Amazon=mzn=14919

In [3]:
def soundex(query: str):
    #Clean up the query string
    #query = query
    letters = [char for char in query if char.isalpha()]
    # If query contains only 1 letter, return query+"000" (Refer step 5)
    if len(query) == 1:
        #if else
        return query + "000"
    to_remove = ('a', 'e', 'i', 'o', 'u', 'y', 'h', 'w')    
    # Save the first letter. Remove all occurrences of a, e, i, o, u, y, h, w.
    first_letter = letters[0]
    #assign 1st char 
    letters = letters[1:]
    # assign rest char
    letters = [char for char in letters if char not in to_remove]
              # (select char from letters where letters not in(select * from to_remove)
    if len(letters) == 0:
        #if else
        return first_letter + "000"
    # Step 2: Replace all consonants (include the first letter) with digits according to rules
    to_replace ={('b'): 1,('p'):2, ('c'):3, ('g'):4, ('j'):5, ('k'):6, ('q'):7, ('s'):8, ('z'): 9,
                   ('d'):11, ('t'):12, ('l'): 13, ('m'):14, ('r',): 15,('f'):16,('v'):17,('x'):18, ('n'): 19}
    first_letter = [value if first_letter else first_letter for group, value in to_replace.items()
                    if first_letter in group]
    print(first_letter)

In [ ]:
def comp_name_std(data_t):
        data_tf=pd.DataFrame()
        #create table data
        for cluster in data_t['code_0'].unique():
            #vinod
            cluster_rows=data_t[data_t['code_0']==cluster]
            cluster_frequency=cluster_rows['Company'].value_counts()
            cluster_frequency=cluster_frequency.to_frame()
            cluster_frequency['comp_name']=cluster_frequency.index
            cluster_frequency['Comp_Name_Std']= cluster_frequency['comp_name'][0]
            data_tf=data_tf.append(cluster_frequency, ignore_index=True)
        return data_tf['Comp_Name_Std'].unique(), data_tf['Comp_Name_Std'].nunique()
    
Step1: company_name_std is a function which takes a table as input called data_t, which has company name "Company", soundex code as "code_0" as columns
Step2: for every unique value in that "code_0", subset the data for that unique values
Step3: in that subset data create frequncy table of all unique Companies through "Company" column.
Step4: assign the company with highest frquency as "Comp_Name_Std"
Step5: append all this data to a table 
Step6: repeat all steps for all unique "code_0" values
       Eg:step1 'Harini','Harini','Haris','xoha',"Zen","Zofa","Zoha" 
          step2  "h123","h123","h123",'z124',"z567","z124"
          step3  "h123" Frequency Table : Harini:2, Harris 1, Hence H123 -> Harini
                 etc
Step7: this function has to run after the soundex function runs 
        
        

In [ ]:
b=pd.DataFrame(a,columns=['std'])

In [292]:
df1=pd.read_csv(r"C:\Users\zafreen\Downloads\R_results.csv")

In [284]:
df1=df1.reset_index(drop=True)

In [ ]:
df1=pd.read_csv("C:/Users/zafreen/Downloads/df1.csv")

In [ ]:
df1=df1[['Company','code_0']]

In [288]:
for i in range(len(df1)):
    print(i)
    df1.loc[i,"code_1"] = soundex(df1['Company'][i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53


IndexError: list index out of range

In [ ]:
df1=df1.reset_index(drop=True)

In [11]:
df1=pd.read_csv(r"C:\Users\zafreen\Downloads\R_results.csv")


In [5]:
# df1.to_csv('R.csv')
# print(df2.shape)
# df1.to_csv("R_soundex.csv")
# df1.head()
# def soundex(query: str):
#     """
#     https://en.wikipedia.org/wiki/Soundex
#     :param query:
#     :return:
#     """

#     # Step 0: Clean up the query string
# #     query = query
#     letters = [char for char in query if char.isalpha()]

#     # Step 1: Save the first letter. Remove all occurrences of a, e, i, o, u, y, h, w.

#     # If query contains only 1 letter, return query+"000" (Refer step 5)
#     if len(query) == 1:
#         return query + "000"

# #    to_remove = ('a','e','i','o','u','y','h','w')


#     first_letter = letters[0]
# #     letters = letters[1:]
# #     letters = [char for char in letters if char not in to_remove]

# #     if len(letters) == 0:
# #         return first_letter + "000"

#     # Step 2: Replace all consonants (include the first letter) with digits according to rules

# #     to_replace = {('b','p'): 1, ('c', 'g', 'j', 'k', 'q', 's', 'z'): 2,
# #                   ('d', 't'): 3, ('l',): 4, ('m'):5, ('r',): 6,('f','v'): 7,( 'x'):8, ('n'): 9}

#     to_replace = {('b', 'f', 'p', 'v'): 1, ('c', 'g', 'j', 'k', 'q', 's', 'x', 'z'): 2,
#                   ('d', 't'): 3, ('l',): 4, ('m', 'n'): 5, ('r',): 6}


#     first_letter = [value if first_letter else first_letter for group, value in to_replace.items()
#                     if first_letter in group]
#     letters = [value if char else char
#                for char in letters
#                for group, value in to_replace.items()
#                if char in group]

#     # Step 3: Replace all adjacent same digits with one digit.
#     letters = [char for ind, char in enumerate(letters)
#                if (ind == len(letters) - 1 or (ind+1 < len(letters) and char != letters[ind+1]))]

#     # Step 4: If the saved letter’s digit is the same the resulting first digit, remove the digit (keep the letter)
#     if first_letter == letters[0]:
#         letters[0] = query[0]
#     else:
#         letters.insert(0, query[0])

#     # Step 5: Append 3 zeros if result contains less than 3 digits.
#     # Remove all except first letter and 3 digits after it.

#     first_letter = letters[0]
#     letters = letters[1:]

#     letters = [char for char in letters if isinstance(char, int)][0:3]

#     while len(letters) < 3:
#         letters.append(0)

#     letters.insert(0, first_letter)

#     string = "".join([str(l) for l in letters])

#     return string

In [12]:
df1.head()

,Unnamed: 0,first,code_1
0,1,iqvia,I210
1,2,infor,I516
2,3,livestyle,L123
3,4,k,K000
4,5,gotjunk,G325


In [13]:
df1=df1.rename(columns={"first":"Company","code_1":"code_0"})
df1=df1[['Company','code_0']]
df1=df1.reset_index(drop=True)

In [14]:
df1.head()

,Company,code_0
0,iqvia,I210
1,infor,I516
2,livestyle,L123
3,k,K000
4,gotjunk,G325


In [14]:
uni,nuni=comp_name_std(df1)

In [16]:
nuni

5327

In [20]:
data_t=df1

In [21]:
data_tf=pd.DataFrame()
for cluster in data_t['code_0'].unique():
    cluster_rows=data_t[data_t['code_0']==cluster]
    cluster_frequency=cluster_rows['Company'].value_counts()
    cluster_frequency=cluster_frequency.to_frame()
    cluster_frequency['comp_name']=cluster_frequency.index
    cluster_frequency['Comp_Name_Std']= cluster_frequency['comp_name'][0]
    data_tf=data_tf.append(cluster_frequency, ignore_index=True)
#  data_tf['Comp_Name_Std'].unique(), data_tf['Comp_Name_Std'].nunique()

In [28]:
data_tf['Comp_/name_Std'].nunique()

KeyError: 'Comp_/name_Std'

In [27]:
# uni=uni.tolist()
pd.DataFrame(data_tf).to_csv("Soundex_uni.csv")

In [178]:
# data_t=df1
data_tf=pd.DataFrame()
li=[]
for cluster in data_t['code_0'].unique():
    cluster_rows=data_t[data_t['code_0']==cluster]
    cluster_frequency=cluster_rows['Company'].value_counts()
    cluster_frequency=cluster_frequency.to_frame()
    cluster_frequency=cluster_frequency.reset_index()
    cluster_frequency=cluster_frequency.rename(columns={'index':'Company Names'})
    cluster_frequency['Comp_Name_Std']= cluster_frequency['Company Names'][0]
    data_tf=data_tf.append(cluster_frequency, ignore_index=True)
    print(cluster_frequency)
   

    

  Company Names  Company Comp_Name_Std
0         neoit        1         neoit
  Company Names  Company Comp_Name_Std
0    nextsource        1    nextsource
  Company Names  Company Comp_Name_Std
0   orthwestern        1   orthwestern
  Company Names  Company Comp_Name_Std
0          stag        1          stag
  Company Names  Company Comp_Name_Std
0        tarang        1        tarang
  Company Names  Company Comp_Name_Std
0    tcognition        1    tcognition
  Company Names  Company Comp_Name_Std
0        telent        1        telent
  Company Names  Company Comp_Name_Std
0  thyssenkrupp        1  thyssenkrupp
  Company Names  Company Comp_Name_Std
0       vsplash        1       vsplash
  Company Names  Company Comp_Name_Std
0   voestalpine        1   voestalpine
  Company Names  Company Comp_Name_Std
0         xwave        1         xwave
  Company Names  Company Comp_Name_Std
0        nergie        1        nergie
  Company Names  Company Comp_Name_Std
0      ekerbank        1 

In [82]:
# d=data_t

In [116]:
df1=df1.reset_index(drop=True)
df1.shape

(9276, 2)

In [180]:
4259+1474+362+150+1930+665+320+103+13

9276

In [177]:
data_t=df1[9263:]
data_t=data_t.reset_index(drop=True)

In [179]:
data_tf.shape

(13, 3)

In [183]:
# data_tf.to_csv("Fist_ENS.csv")
# data_tf.to_csv("Second.ENS.csv")
# data_tf.to_csv("Third_ENS.csv")
# data_tf.to_csv("Fourth_ENS.csv")
# data_tf.to_csv("Five_ENS.csv")
# data_tf.to_csv("Sixth_ENS.csv")
# data_tf.to_csv("Seventh_ENS.csv")
# data_tf.to_csv("Eight_ENS.csv")
# data_tf.to_csv("Ninth_ENS.csv")
d1=pd.read_csv(r"C:\Users\zafreen\Downloads\ENS\Code_ENS\Fist_ENS.csv")
d2=pd.read_csv(r"C:\Users\zafreen\Downloads\ENS\Code_ENS\Second.ENS.csv")
d3=pd.read_csv(r"C:\Users\zafreen\Downloads\ENS\Code_ENS\Third_ENS.csv")
d4=pd.read_csv(r"C:\Users\zafreen\Downloads\ENS\Code_ENS\Fourth_ENS.csv")
d5=pd.read_csv(r"C:\Users\zafreen\Downloads\ENS\Code_ENS\Five_ENS.csv")
d6=pd.read_csv(r"C:\Users\zafreen\Downloads\ENS\Code_ENS\Sixth_ENS.csv")
d7=pd.read_csv(r"C:\Users\zafreen\Downloads\ENS\Code_ENS\Seventh_ENS.csv")
d8=pd.read_csv(r"C:\Users\zafreen\Downloads\ENS\Code_ENS\Eight_ENS.csv")
d9=pd.read_csv(r"C:\Users\zafreen\Downloads\ENS\Code_ENS\Ninth_ENS.csv")



In [184]:
data=pd.concat([d1,d2,d3,d4,d5,d6,d7,d8,d9])
data=data.reset_index(drop=True)
data.shape

(9276, 4)

In [197]:
a=data['Comp_Name_Std'].unique()
a.tolist()
d1=pd.DataFrame({'Unique Companies':a})
d1.to_csv("Unique Companies.csv")

In [301]:
def comp_name_std(data_t):
        a=0
        data_tf=pd.DataFrame()
        for cluster in data_t['code_0'].unique():
            cluster_rows=data_t[data_t['code_0']==cluster]
            cluster_frequency=cluster_rows['Company'].value_counts()
            cluster_frequency=cluster_frequency.to_frame()
            cluster_frequency['comp_name']=cluster_frequency.index
            print(cluster_frequency)
            cluster_frequency['Comp_Name_Std']= cluster_frequency['comp_name'][0]
            data_tf=data_tf.append(cluster_frequency, ignore_index=True)
        return data_tf

In [8]:
new=comp_name_std(df1)

NameError: name 'comp_name_std' is not defined

In [ ]:
## Actual number of clusters = 76
# Website Algorithm clustered = 19 clusters
# Our Algorithm clustered = 57 clusters

In [ ]:
# 1) Pckages 
# 2) data and cleaning 
# 3) proof of 19 companies only 
# 4) Our algorithm and proof of 57 comanies - gt the external hard drive 
# Deadline 3 PM 